In [433]:
# author: Jerry Tsai
# program gbrt_eda.ipynb
# creation date: 2016-05-03
# version 1.0
#
# PURPOSE: Make several analysis data sets from the data in the
# Upwork provider profiles data (text file)
#

import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import time
import datetime
import re
import cPickle as pickle

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble.partial_dependence import plot_partial_dependence
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

In [434]:
### MAKE PICKLES OF NEW DATAFRAMES
infile_skills_vectorized = 'data/_skills_vectorized_df.pkl'
infile_profiles = 'data/profiles_df.pkl'
infile_jobs = 'data/jobs_df.pkl'
infile_hr_paid_jobs = 'data/hr_paid_jobs_df.pkl'
infile_skills = 'data/skills_df.pkl'

hr_paid_jobs_df = pickle.load(open(infile_hr_paid_jobs, 'rb'))
sv_df = pickle.load(open(infile_skills_vectorized, 'rb'))
profiles_df = pickle.load(open(infile_profiles, 'rb'))

## Work up `profiles_df`

In [435]:
profiles_df.T.head(30)

0      \
dev_bill_rate                                                        15   
dev_blurb             Over the more than 6 years, I have developed a...   
dev_country                                                       India   
dev_eng_skill                                                         4   
dev_portrait          https://odesk-prod-portraits.s3.amazonaws.com/...   
dev_profile_title                 LAMP developer with more than 6 years   
dev_recno_ciphertext                                ~01cae3810dc6d2bbbc   
dev_timezone              UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi   
education             {u'institution': {u'ed_to': u'07/2007', u'ed_a...   
agency_affl                                                       False   
group_affl                                                        False   
has_portrait                                                       True   
highest_degree                                                  Masters   
has_portfolio                                                      True   
took_tests                                                         True   
skills_string         ajax apidashdevelopment curl javascript magent...   

                                                                  1      \
dev_bill_rate                                                        12   
dev_blurb             I am a web application developer, having 2 yea...   
dev_country                                                       India   
dev_eng_skill                                                         5   
dev_portrait          https://odesk-prod-portraits.s3.amazonaws.com/...   
dev_profile_title                              Web Developer / Designer   
dev_recno_ciphertext                                ~01d24ced467385214d   
dev_timezone              UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi   
education             {u'institution': {u'ed_to': u'12/2012', u'ed_a...   
agency_affl                                                       False   
group_affl                                                        False   
has_portrait                                                       True   
highest_degree                         Bachelor of Engineering (B.Eng.)   
has_portfolio                                                     False   
took_tests                                                         True   
skills_string         ajax css3 grails groovy javascript jquery mysq...   

                                                                  2      \
dev_bill_rate                                                     44.44   
dev_blurb             Extensive expertise in clinical research publi...   
dev_country                                               United States   
dev_eng_skill                                                         5   
dev_portrait          https://odesk-prod-portraits.s3.amazonaws.com/...   
dev_profile_title     English/French Writer/Translator,Scientific Di...   
dev_recno_ciphertext                                ~01953fb4fa2ceb67dc   
dev_timezone          UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels   
education             {u'institution': [{u'ed_to': u'12/2002', u'ed_...   
agency_affl                                                       False   
group_affl                                                        False   
has_portrait                                                       True   
highest_degree               Master of Business Administration (M.B.A.)   
has_portfolio                                                     False   
took_tests                                                         True   
skills_string         articledashwriting creativedashwriting datadas...   

                                                                  3      \
dev_bill_rate                                                     24.44   
dev_blurb             I'm a WEB-developer with more than 5 years exp...   
dev_country  

In [473]:
profiles_usa_and_canada_ids = profiles_df[ (profiles_df['dev_country']=='United States') | \
                                      (profiles_df['dev_country']=='Canada') ]['dev_recno_ciphertext'].copy()

In [441]:
profiles_usa_and_canada_ids.head()

2     ~01953fb4fa2ceb67dc
8     ~0122260b02b0c56563
10    ~018177de0182ba87ad
22    ~01fff0a5f35f1c6ac7
28    ~015f3926e293b176df
Name: dev_recno_ciphertext, dtype: object

## Work up `sv_df`

In [442]:
svcopy_df = sv_df.copy()

In [181]:
sv_df = svcopy_df.copy()

In [182]:
# print np.array(sv_df.columns)[sums > 1000]

In [443]:
skills_sums = np.array(sv_df.sum(axis=0))
pct_threshold = 0.5 / 100.
threshold = pct_threshold*float(len(sv_df))
insufficient_data_columns = (skills_sums < threshold)

In [444]:
print '# of columns that will be kept: {}'.format(len(skills_sums) - np.sum(insufficient_data_columns))

# of columns that will be kept: 219


In [445]:
columns_to_drop = list(np.arange(len(skills_sums))[insufficient_data_columns])
sv_df.drop(sv_df.columns[columns_to_drop], axis=1, inplace=True)

In [447]:
sv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88696 entries, 0 to 88695
Columns: 219 entries, accounting to zenddashframework
dtypes: int64(219)
memory usage: 148.9 MB


## Graft `sv_df` to `profiles_df` to create `profile_anly_ds`

In [448]:
profile_anly_ds = pd.concat([profiles_df['dev_recno_ciphertext'], sv_df], axis=1)

## Work up `hr_paid_jobs_df`

In [469]:
def get_number(str):
    return float(re.search(r'\d+.*\d*', str).group())

hr_paid_jobs_df['hours_as_number'] = hr_paid_jobs_df['as_total_hours_precise'].apply(get_number)
hr_paid_jobs_df['sqrt_hours'] = (hr_paid_jobs_df['hours_as_number'])**(0.5)

In [470]:
todays_date = datetime.datetime.now().strftime('%m/%d/%Y')

In [471]:
hr_paid_jobs_df['end_date_str'] = hr_paid_jobs_df['as_to_full'].replace({'Present': todays_date})

In [472]:
hr_paid_jobs_df['end_date'] = pd.to_datetime(hr_paid_jobs_df['end_date_str'])

In [474]:
hr_paid_jobs_df.drop([ \
    'end_date_str', \
    'as_from_full', \
    'as_to_full', \
    'as_rate', \
    'as_total_charge', \
    'as_total_hours_precise' \
], axis=1, inplace=True)

In [476]:
hr_paid_jobs_2013_plus = hr_paid_jobs_df[hr_paid_jobs_df['end_date'] > datetime.datetime(2013, 1, 1, 0, 0, 0)].copy()

In [477]:
hr_paid_jobs_2013_plus.drop(['end_date', 'hours_as_number'], axis=1, inplace=True)

In [478]:
hr_paid_jobs_2013_plus.head()

dev_recno_ciphertext  rate_as_number  sqrt_hours
1   ~01a9ab111f973ae055           22.22   24.779023
2   ~01a9ab111f973ae055           16.67   34.578413
13  ~0122260b02b0c56563            3.33    1.527525
27  ~01df9ed93f98f6f65e            8.89    2.645751
32  ~018702fd0c3029a744           16.67    2.041241

## Create analysis data set for Questions 1 and 2 (GBRT)

In [491]:
jobs_anly_temp = pd.merge( \
                        left=profile_anly_ds, \
                        right=hr_paid_jobs_2013_plus, \
                        how='inner', left_on='dev_recno_ciphertext', \
                        right_on='dev_recno_ciphertext' \
                       )

In [492]:
jobs_anly_ds= pd.merge( \
                        left=pd.DataFrame(profiles_usa_and_canada_ids), \
                        right=jobs_anly_temp, \
                        how='inner', left_on='dev_recno_ciphertext', \
                        right_on='dev_recno_ciphertext' \
                       )

In [493]:
jobs_anly_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2506 entries, 0 to 2505
Columns: 222 entries, dev_recno_ciphertext to sqrt_hours
dtypes: float64(2), int64(219), object(1)
memory usage: 4.3+ MB


## Run GBRT models

### Try one GBRT model

In [494]:
target = 'rate_as_number'
IDcol = 'dev_recno_ciphertext'
weight = 'sqrt_hours'
predictors = [x for x in jobs_anly_ds.columns if x not in [target, IDcol, weight]]

y = jobs_anly_ds.pop('rate_as_number').values
weights = jobs_anly_ds.pop('sqrt_hours').values
X = jobs_anly_ds[predictors].values

In [495]:
with open('data/predictors.txt', 'w') as f:
    for pred in predictors:
        f.write(pred + '\n')

In [498]:
#Choose all predictors except target & IDcols
gbm0 = GradientBoostingRegressor(random_state=8, n_estimators=1000, learning_rate=0.1)
gbm0.fit(X, y, weights)
mse = mean_squared_error(y, gbm0.predict(X), sample_weight=weights)
print mse


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=8, subsample=1.0, verbose=0,
             warm_start=False)

In [501]:
np_predictors = np.array(predictors)
skill_set = ( \
             (np_predictors == 'python') + \
             (np_predictors == 'statistics') + \
             (np_predictors == 'machinedashlearning') \
            ).reshape(1, -1)
print np.sum(skill_set)

3


In [502]:
gbm0.predict(skill_set)

array([ 36.67513496])

In [503]:
matrix_side = len(np_predictors)
skill_set_for_all = np.repeat(skill_set, matrix_side, axis=0)
plus_one_skill = (skill_set_for_all) | (np.eye(matrix_side).astype(bool))

In [504]:
plus_one_preds = gbm0.predict(plus_one_skill)
top_20_plus_ones = plus_one_preds.argsort()[-20:][::-1]
print plus_one_preds[top_20_plus_ones ] - gbm0.predict(skill_set)
print np_predictors[top_20_plus_ones]

[ 56.23859484  20.36099434  18.2140494   14.43088875  13.5926924
  13.19839603  11.75510666  10.41121274   9.83235662   9.00231807
   8.9528251    8.38389016   7.93531294   7.7812883    6.97937318
   6.5781361    6.00121592   5.99349842   5.78446369   5.75584615]
[u'jpa' u'datadashscience' u'mysql' u'databasedashmodeling' u'bigdashdata'
 u'dotnetdashframework' u'oracledashplsql' u'datadashmodeling'
 u'datadashmining' u'iphonedashappdashdevelopment' u'wcf'
 u'webdashprogramming' u'networkdashadministration'
 u'amazondashwebdashservices' u'jquerydashmobile' u'matlab' u'scala'
 u'mobiledashappdashdevelopment' u'mysqldashprogramming'
 u'microsoftdashsqldashssrs']


## Optimize regression model

In [505]:
gd_grid = {'learning_rate': [0.06, 0.04, 0.02],
           'max_depth': [4, 6],
           'min_samples_leaf': [10],
           'n_estimators': [300, 500, 800],
           'random_state': [1]}

In [506]:
results_list_of_tuples = list()
num_folds = 3
best_result = tuple()
for item1 in gd_grid['learning_rate']:
    for item2 in gd_grid['max_depth']:
        for item3 in gd_grid['min_samples_leaf']:
            for item4 in gd_grid['n_estimators']:
                for item5 in gd_grid['random_state']:
                    instance = \
                      'LR {}, max_depth {}, min_samp_leaf {}, n_est {}, rs {}'.format(item1, item2, item3, item4, item5)
                    print instance
                    gbrt = GradientBoostingRegressor(random_state=item5, \
                                                     n_estimators=item4, \
                                                     min_samples_leaf=item3, \
                                                     max_depth=item2, \
                                                     learning_rate=item1 \
                                                    )
                    kf = KFold(X.shape[0], n_folds=num_folds)
                    mse_list = []
                    for train_index, test_index in kf:
                        X_train, X_test = X[train_index], X[test_index]
                        y_train, y_test = y[train_index], y[test_index]
                        w_train, w_test = weights[train_index], weights[test_index]
                        gbrt.fit(X_train, y_train, w_train)                 
                        y_pred = gbrt.predict(X_test)
                        mse = mean_squared_error(y_test, y_pred, sample_weight=w_test)
                        mse_list.append(mse)

                    kf_mse = np.mean(np.array(mse_list))
                    results_list_of_tuples.append((instance, kf_mse))



LR 0.06, max_depth 4, min_samp_leaf 10, n_est 300, rs 1
LR 0.06, max_depth 4, min_samp_leaf 10, n_est 500, rs 1
LR 0.06, max_depth 4, min_samp_leaf 10, n_est 800, rs 1
LR 0.06, max_depth 6, min_samp_leaf 10, n_est 300, rs 1
LR 0.06, max_depth 6, min_samp_leaf 10, n_est 500, rs 1
LR 0.06, max_depth 6, min_samp_leaf 10, n_est 800, rs 1
LR 0.04, max_depth 4, min_samp_leaf 10, n_est 300, rs 1
LR 0.04, max_depth 4, min_samp_leaf 10, n_est 500, rs 1
LR 0.04, max_depth 4, min_samp_leaf 10, n_est 800, rs 1
LR 0.04, max_depth 6, min_samp_leaf 10, n_est 300, rs 1
LR 0.04, max_depth 6, min_samp_leaf 10, n_est 500, rs 1
LR 0.04, max_depth 6, min_samp_leaf 10, n_est 800, rs 1
LR 0.02, max_depth 4, min_samp_leaf 10, n_est 300, rs 1
LR 0.02, max_depth 4, min_samp_leaf 10, n_est 500, rs 1
LR 0.02, max_depth 4, min_samp_leaf 10, n_est 800, rs 1
LR 0.02, max_depth 6, min_samp_leaf 10, n_est 300, rs 1
LR 0.02, max_depth 6, min_samp_leaf 10, n_est 500, rs 1
LR 0.02, max_depth 6, min_samp_leaf 10, n_est 80

In [507]:
for tup in results_list_of_tuples:
    print tup

('LR 0.06, max_depth 4, min_samp_leaf 10, n_est 300, rs 1', 453.3626821205894)
('LR 0.06, max_depth 4, min_samp_leaf 10, n_est 500, rs 1', 465.48144835118501)
('LR 0.06, max_depth 4, min_samp_leaf 10, n_est 800, rs 1', 475.99934640327848)
('LR 0.06, max_depth 6, min_samp_leaf 10, n_est 300, rs 1', 450.13242856614937)
('LR 0.06, max_depth 6, min_samp_leaf 10, n_est 500, rs 1', 461.87015950912746)
('LR 0.06, max_depth 6, min_samp_leaf 10, n_est 800, rs 1', 473.05131934775426)
('LR 0.04, max_depth 4, min_samp_leaf 10, n_est 300, rs 1', 445.31174758719317)
('LR 0.04, max_depth 4, min_samp_leaf 10, n_est 500, rs 1', 452.46870537856086)
('LR 0.04, max_depth 4, min_samp_leaf 10, n_est 800, rs 1', 460.03906030353761)
('LR 0.04, max_depth 6, min_samp_leaf 10, n_est 300, rs 1', 441.28199553078042)
('LR 0.04, max_depth 6, min_samp_leaf 10, n_est 500, rs 1', 456.22120358768854)
('LR 0.04, max_depth 6, min_samp_leaf 10, n_est 800, rs 1', 462.17799325050009)
('LR 0.02, max_depth 4, min_samp_leaf 10,

In [508]:
gd_grid = {'learning_rate': [0.024, 0.016, 0.008],
           'max_depth': [5, 6, 8],
           'min_samples_leaf': [10],
           'n_estimators': [240, 300, 360],
           'random_state': [1]}

In [509]:
def grid_search():
    results_list_of_tuples = list()
    num_folds = 3
    best_result = tuple()
    for item1 in gd_grid['learning_rate']:
        for item2 in gd_grid['max_depth']:
            for item3 in gd_grid['min_samples_leaf']:
                for item4 in gd_grid['n_estimators']:
                    for item5 in gd_grid['random_state']:
                        instance = \
                          'LR {}, max_depth {}, min_samp_leaf {}, n_est {}, rs {}'.format(item1, \
                                                                                          item2, item3, \
                                                                                          item4, item5)
                        print instance
                        gbrt = GradientBoostingRegressor(random_state=item5, \
                                                         n_estimators=item4, \
                                                         min_samples_leaf=item3, \
                                                         max_depth=item2, \
                                                         learning_rate=item1 \
                                                        )
                        kf = KFold(X.shape[0], n_folds=num_folds)
                        mse_list = []
                        for train_index, test_index in kf:
                            X_train, X_test = X[train_index], X[test_index]
                            y_train, y_test = y[train_index], y[test_index]
                            w_train, w_test = weights[train_index], weights[test_index]
                            gbrt.fit(X_train, y_train, w_train)                 
                            y_pred = gbrt.predict(X_test)
                            mse = mean_squared_error(y_test, y_pred, sample_weight=w_test)
                            mse_list.append(mse)

                        kf_mse = np.mean(np.array(mse_list))
                        results_list_of_tuples.append((instance, kf_mse))
                        
    return results_list_of_tuples

grid_search()

LR 0.06, max_depth 5, min_samp_leaf 10, n_est 240, rs 1
LR 0.06, max_depth 5, min_samp_leaf 10, n_est 300, rs 1
LR 0.06, max_depth 5, min_samp_leaf 10, n_est 360, rs 1
LR 0.06, max_depth 6, min_samp_leaf 10, n_est 240, rs 1
LR 0.06, max_depth 6, min_samp_leaf 10, n_est 300, rs 1
LR 0.06, max_depth 6, min_samp_leaf 10, n_est 360, rs 1
LR 0.06, max_depth 8, min_samp_leaf 10, n_est 240, rs 1
LR 0.06, max_depth 8, min_samp_leaf 10, n_est 300, rs 1
LR 0.06, max_depth 8, min_samp_leaf 10, n_est 360, rs 1
LR 0.024, max_depth 5, min_samp_leaf 10, n_est 240, rs 1
LR 0.024, max_depth 5, min_samp_leaf 10, n_est 300, rs 1
LR 0.024, max_depth 5, min_samp_leaf 10, n_est 360, rs 1
LR 0.024, max_depth 6, min_samp_leaf 10, n_est 240, rs 1
LR 0.024, max_depth 6, min_samp_leaf 10, n_est 300, rs 1
LR 0.024, max_depth 6, min_samp_leaf 10, n_est 360, rs 1
LR 0.024, max_depth 8, min_samp_leaf 10, n_est 240, rs 1
LR 0.024, max_depth 8, min_samp_leaf 10, n_est 300, rs 1
LR 0.024, max_depth 8, min_samp_leaf 10,

In [510]:
for tup in results_list_of_tuples:
    print tup

('LR 0.06, max_depth 5, min_samp_leaf 10, n_est 240, rs 1', 442.67127209635902)
('LR 0.06, max_depth 5, min_samp_leaf 10, n_est 300, rs 1', 449.00054244082804)
('LR 0.06, max_depth 5, min_samp_leaf 10, n_est 360, rs 1', 454.64603716545071)
('LR 0.06, max_depth 6, min_samp_leaf 10, n_est 240, rs 1', 449.14732191899066)
('LR 0.06, max_depth 6, min_samp_leaf 10, n_est 300, rs 1', 450.13242856614937)
('LR 0.06, max_depth 6, min_samp_leaf 10, n_est 360, rs 1', 452.56121816958665)
('LR 0.06, max_depth 8, min_samp_leaf 10, n_est 240, rs 1', 442.24173921022452)
('LR 0.06, max_depth 8, min_samp_leaf 10, n_est 300, rs 1', 444.53967375418603)
('LR 0.06, max_depth 8, min_samp_leaf 10, n_est 360, rs 1', 448.12675872922131)
('LR 0.024, max_depth 5, min_samp_leaf 10, n_est 240, rs 1', 434.35863921539863)
('LR 0.024, max_depth 5, min_samp_leaf 10, n_est 300, rs 1', 434.1193734365267)
('LR 0.024, max_depth 5, min_samp_leaf 10, n_est 360, rs 1', 435.00130352179758)
('LR 0.024, max_depth 6, min_samp_leaf

In [512]:
gd_grid = {'learning_rate': [0.008, 0.004, 0.002],
           'max_depth': [6],
           'min_samples_leaf': [7, 10, 13],
           'n_estimators': [280, 300, 320],
           'random_state': [2]}

In [513]:
def grid_search():
    results_list_of_tuples = list()
    num_folds = 3
    best_result = tuple()
    for item1 in gd_grid['learning_rate']:
        for item2 in gd_grid['max_depth']:
            for item3 in gd_grid['min_samples_leaf']:
                for item4 in gd_grid['n_estimators']:
                    for item5 in gd_grid['random_state']:
                        instance = \
                          'LR {}, max_depth {}, min_samp_leaf {}, n_est {}, rs {}'.format(item1, \
                                                                                          item2, item3, \
                                                                                          item4, item5)
                        print instance
                        gbrt = GradientBoostingRegressor(random_state=item5, \
                                                         n_estimators=item4, \
                                                         min_samples_leaf=item3, \
                                                         max_depth=item2, \
                                                         learning_rate=item1 \
                                                        )
                        kf = KFold(X.shape[0], n_folds=num_folds)
                        mse_list = []
                        for train_index, test_index in kf:
                            X_train, X_test = X[train_index], X[test_index]
                            y_train, y_test = y[train_index], y[test_index]
                            w_train, w_test = weights[train_index], weights[test_index]
                            gbrt.fit(X_train, y_train, w_train)                 
                            y_pred = gbrt.predict(X_test)
                            mse = mean_squared_error(y_test, y_pred, sample_weight=w_test)
                            mse_list.append(mse)

                        kf_mse = np.mean(np.array(mse_list))
                        results_list_of_tuples.append((instance, kf_mse))
                        
    return results_list_of_tuples

print grid_search()

LR 0.008, max_depth 6, min_samp_leaf 7, n_est 280, rs 2
LR 0.008, max_depth 6, min_samp_leaf 7, n_est 300, rs 2
LR 0.008, max_depth 6, min_samp_leaf 7, n_est 320, rs 2
LR 0.008, max_depth 6, min_samp_leaf 10, n_est 280, rs 2
LR 0.008, max_depth 6, min_samp_leaf 10, n_est 300, rs 2
LR 0.008, max_depth 6, min_samp_leaf 10, n_est 320, rs 2
LR 0.008, max_depth 6, min_samp_leaf 13, n_est 280, rs 2
LR 0.008, max_depth 6, min_samp_leaf 13, n_est 300, rs 2
LR 0.008, max_depth 6, min_samp_leaf 13, n_est 320, rs 2
LR 0.004, max_depth 6, min_samp_leaf 7, n_est 280, rs 2
LR 0.004, max_depth 6, min_samp_leaf 7, n_est 300, rs 2
LR 0.004, max_depth 6, min_samp_leaf 7, n_est 320, rs 2
LR 0.004, max_depth 6, min_samp_leaf 10, n_est 280, rs 2
LR 0.004, max_depth 6, min_samp_leaf 10, n_est 300, rs 2
LR 0.004, max_depth 6, min_samp_leaf 10, n_est 320, rs 2
LR 0.004, max_depth 6, min_samp_leaf 13, n_est 280, rs 2
LR 0.004, max_depth 6, min_samp_leaf 13, n_est 300, rs 2
LR 0.004, max_depth 6, min_samp_leaf 

[('LR 0.008, max_depth 6, min_samp_leaf 7, n_est 280, rs 2',
  440.39592700347265),
 ('LR 0.008, max_depth 6, min_samp_leaf 7, n_est 300, rs 2',
  440.43655065612535),
 ('LR 0.008, max_depth 6, min_samp_leaf 7, n_est 320, rs 2',
  440.2126389186144),
 ('LR 0.008, max_depth 6, min_samp_leaf 10, n_est 280, rs 2',
  427.47015348479528),
 ('LR 0.008, max_depth 6, min_samp_leaf 10, n_est 300, rs 2',
  427.53954278821857),
 ('LR 0.008, max_depth 6, min_samp_leaf 10, n_est 320, rs 2',
  427.41569234005402),
 ('LR 0.008, max_depth 6, min_samp_leaf 13, n_est 280, rs 2',
  424.57414037312736),
 ('LR 0.008, max_depth 6, min_samp_leaf 13, n_est 300, rs 2',
  424.45884207251601),
 ('LR 0.008, max_depth 6, min_samp_leaf 13, n_est 320, rs 2',
  424.47116017713842),
 ('LR 0.004, max_depth 6, min_samp_leaf 7, n_est 280, rs 2',
  437.43214110972286),
 ('LR 0.004, max_depth 6, min_samp_leaf 7, n_est 300, rs 2',
  437.74764622363699),
 ('LR 0.004, max_depth 6, min_samp_leaf 7, n_est 320, rs 2',
  438.3365

In [515]:
gd_grid = {'learning_rate': [0.008, 0.007],
           'max_depth': [6],
           'min_samples_leaf': [13, 16, 21],
           'n_estimators': [300],
           'random_state': [2]}

print grid_search()

LR 0.008, max_depth 6, min_samp_leaf 13, n_est 300, rs 2
LR 0.008, max_depth 6, min_samp_leaf 16, n_est 300, rs 2
LR 0.008, max_depth 6, min_samp_leaf 21, n_est 300, rs 2
LR 0.007, max_depth 6, min_samp_leaf 13, n_est 300, rs 2
LR 0.007, max_depth 6, min_samp_leaf 16, n_est 300, rs 2
LR 0.007, max_depth 6, min_samp_leaf 21, n_est 300, rs 2
[('LR 0.008, max_depth 6, min_samp_leaf 13, n_est 300, rs 2', 424.45884207251601), ('LR 0.008, max_depth 6, min_samp_leaf 16, n_est 300, rs 2', 428.16268222169418), ('LR 0.008, max_depth 6, min_samp_leaf 21, n_est 300, rs 2', 435.10303325669469), ('LR 0.007, max_depth 6, min_samp_leaf 13, n_est 300, rs 2', 424.5705162139659), ('LR 0.007, max_depth 6, min_samp_leaf 16, n_est 300, rs 2', 427.70488060600809), ('LR 0.007, max_depth 6, min_samp_leaf 21, n_est 300, rs 2', 434.93540142861229)]


In [517]:
gbm1 = GradientBoostingRegressor( \
                                 learning_rate=0.008, \
                                 max_depth=6, \
                                 min_samples_leaf=13, \
                                 n_estimators=300, \
                                 random_state=3, \
                                )
gbm1.fit(X, y, weights)

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.008,
             loss='ls', max_depth=6, max_features=None,
             max_leaf_nodes=None, min_samples_leaf=13, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             presort='auto', random_state=3, subsample=1.0, verbose=0,
             warm_start=False)

In [518]:
np_predictors = np.array(predictors)
skill_set = ( \
             (np_predictors == 'python') + \
             (np_predictors == 'statistics') + \
             (np_predictors == 'machinedashlearning') \
            ).reshape(1, -1)
print np.sum(skill_set)

3


In [520]:
gbm1.predict(skill_set)

array([ 33.66257463])

In [521]:
matrix_side = len(np_predictors)
skill_set_for_all = np.repeat(skill_set, matrix_side, axis=0)
plus_one_skill = (skill_set_for_all) | (np.eye(matrix_side).astype(bool))

In [526]:
plus_one_preds = gbm0.predict(plus_one_skill)
top_20_plus_ones = plus_one_preds.argsort()[-20:][::-1]
delta = plus_one_preds[top_20_plus_ones ] - gbm0.predict(skill_set)
print delta
print np_predictors[top_20_plus_ones]

[ 56.23859484  20.36099434  18.2140494   14.43088875  13.5926924
  13.19839603  11.75510666  10.41121274   9.83235662   9.00231807
   8.9528251    8.38389016   7.93531294   7.7812883    6.97937318
   6.5781361    6.00121592   5.99349842   5.78446369   5.75584615]
[u'jpa' u'datadashscience' u'mysql' u'databasedashmodeling' u'bigdashdata'
 u'dotnetdashframework' u'oracledashplsql' u'datadashmodeling'
 u'datadashmining' u'iphonedashappdashdevelopment' u'wcf'
 u'webdashprogramming' u'networkdashadministration'
 u'amazondashwebdashservices' u'jquerydashmobile' u'matlab' u'scala'
 u'mobiledashappdashdevelopment' u'mysqldashprogramming'
 u'microsoftdashsqldashssrs']


In [530]:
for tup in zip(delta, 100*delta/gbm1.predict(skill_set), np_predictors[top_20_plus_ones]):
    print tup

(56.238594835769447, 167.06563729551354, u'jpa')
(20.360994342687263, 60.485552773232854, u'datadashscience')
(18.214049399948564, 54.107713388297, u'mysql')
(14.430888747915155, 42.869236552789758, u'databasedashmodeling')
(13.592692400221708, 40.379241782914889, u'bigdashdata')
(13.198396032523831, 39.20792208431434, u'dotnetdashframework')
(11.755106664081353, 34.92040283094709, u'oracledashplsql')
(10.411212743291088, 30.928153469275777, u'datadashmodeling')
(9.8323566216681897, 29.208569842697852, u'datadashmining')
(9.0023180695058116, 26.742809094197533, u'iphonedashappdashdevelopment')
(8.9528250961076523, 26.595782391866869, u'wcf')
(8.3838901628878446, 24.905671223982687, u'webdashprogramming')
(7.9353129401951108, 23.573101663803211, u'networkdashadministration')
(7.7812882963371948, 23.115547107888691, u'amazondashwebdashservices')
(6.9793731833287467, 20.733331481717858, u'jquerydashmobile')
(6.5781361008416468, 19.541393292507184, u'matlab')
(6.0012159182218099, 17.827560

In [ ]:
skill_set = ( \
             (np_predictors == 'python') + \
             (np_predictors == 'statistics') + \
             (np_predictors == 'machinedashlearning') \
            ).reshape(1, -1)

In [534]:
skill_set_list = [
    'python',
    'statistics',
    'machinedashlearning'
]

In [536]:
skill_index_list = [predictors.index(skill) for skill in skill_set_list]
print skill_index_list

[156, 185, 110]


In [542]:
plus_one_list = [ \
    'datadashmining', \
    'bigdashdata', \
    'datadashscience', \
    'jpa' \
]

In [543]:
plus_one_index_list = [predictors.index(skill) for skill in plus_one_list]
print plus_one_index_list

[57, 30, 59, 100]


In [556]:
def distance_metric(skill_index_list, plus_one_index):
    index_array = np.array([plus_one_index] + skill_index_list)
    calc_X = X[:, index_array]
    print calc_X.shape
    only_plus_one_rows = calc_X[calc_X[:, 0] == 1,]
    distance = 1 - (np.sum(np.sum(only_plus_one_rows, axis=1) > 1)/float(only_plus_one_rows.shape[0]))
    return distance

In [557]:
for plus_one_index in plus_one_index_list:
    print predictors[plus_one_index], distance_metric(skill_index_list, plus_one_index)

datadashmining (2506, 4)
0.60248447205
bigdashdata (2506, 4)
0.769230769231
datadashscience (2506, 4)
0.163636363636
jpa (2506, 4)
1.0
